In [19]:
import qsharp

In [20]:
%%qsharp


//namespace QAOA {



    open Microsoft.Quantum.Canon;
    open Microsoft.Quantum.Intrinsic;
    open Microsoft.Quantum.Convert;
    open Microsoft.Quantum.Math;
    open Microsoft.Quantum.Arithmetic;
    open Microsoft.Quantum.Preparation;
    open Microsoft.Quantum.Diagnostics;




    //QAOA circuit loop
    @EntryPoint()
    operation QAOA (p:Int, beta:Double, gamma:Double) : Int[]{

        //input qubits
        use register = Qubit[Length(measure)];

        //prepare qubits by creating uniform superposition
        ApplyToEach(H, register);

        //repeat cost and mixer steps p times
        //I think that we might need a new gamma and beta for each p
        for i in 1..p{
            costHamiltonianFlexible(register, gamma);
            mixerHamiltonian(register, beta);
        }

        //Read the results as a boolean array (true==1, false==0)
        mutable res = new Int[Length(register)];
        for i in 0..Length(register){
            set res w/= i <- (M(register[i]) == One ? 1 | 0);
        }

        //return the results
        return res;
     }

     operation costHamiltonianLength5 (register:Qubit[], gamma:Double):Unit{
        //Assuming lenght of register is 5
        //I aint commenting this sheet
        let (q0, q1, q2, q3, q4, q5)= (register[0], register[1], register[2], register[3], register[4], register[5]);
        CNOT(q1, q0);
        Rz(gamma, q0);
        CNOT(q1, q0);
        CNOT(q2, q0);
        CNOT(q2, q0);
        Rz(gamma, q0);
        CNOT(q2, q0);
        CNOT(q2, q1);
        CNOT(q3, q0);
        Rz(5.*gamma, q0);
        Rz(2.*gamma, q1);
        CNOT(q2, q1);
        CNOT(q3, q0);
        CNOT(q3, q1);
        CNOT(q4, q0);
        Rz(5.*gamma, q0);
        Rz(gamma, q1);
        CNOT(q3, q1);
        CNOT(q4, q0);
        CNOT(q3, q2);
        CNOT(q4, q1);
        Rz(3.*gamma, q1);
        Rz(5.*gamma, q2);
        CNOT(q3, q2);
        CNOT(q4, q1);
        CNOT(q4, q2);
        Rz(gamma, q2);
        CNOT(q4, q2);
        CNOT(q4, q3);
        Rz(5.*gamma, q3);
        CNOT(q4, q3);
     }

     //Flexible cost hamiltonian layer
     //Misssing weights
     operation costHamiltonianFlexible (register : Qubit[], gamma:Double):Unit{         
         ApplyToEach(Rx(gamma, _), register);
         for i in 0..Length(register){
             for j in i+1..Length(register){
                 Rzz(gamma, register[i], register[j]);
             }
         }
    }

     operation mixerHamiltonian (register:Qubit[], beta:Double):Unit{
         //uhhh yeah you just like do this
         ApplyToEach(Rx(2.*beta, _), register);
     }

     //To be implemented
    operation cost (x:Int):Double{
        return 0.0;
    }
    
    operation meanCost(x:Int[]):Double{
        mutable res = 0.0;
        for i in 0..Length(x){
            set res += cost(x[i]);
        }
        return res/IntAsDouble(Length(x));
    }

    operation classicalOptimizerBasic(meanCostRes:Double, gamma:Double, beta:Double, p:Int):(Double, Double){
        mutable g = gamma;
        mutable b = beta;
        mutable min = [gamma, beta];
        for i in -1..1{
            if i !=0{
                for j in -1..1{
                    if j!= 0{
                        set g = gamma + IntAsDouble(i);
                        set b = beta + IntAsDouble(j);
                        if meanCostRes < meanCost(QAOA(p, g, b)){
                            set min = [g, b];
                        } 
                    }
                }
            }
        }
        return (min[0], min[1]);
    }
//}

D:\snippet_.qs(36,23): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [21]:
import numpy as np
from scipy.optimize import minimize

In [45]:
#def obj_func(angles):
#    gamma, beta = angles
#    return meanCost.simulate(x=QAOA.simulate(measure=[true, true, true, true, true], p=1, g=gamma, b=beta))

def obj_func(params, *args):
    return meanCost.simulate(params, *args)

# Initial guess for the parameters
initial_guess = [0.0, 0.0]

args = (1,)

# Perform optimization
result = minimize(obj_func, initial_guess, args=args )

# Extract the optimized parameters
optimized_params = result.x

TypeError: simulate() takes 1 positional argument but 3 were given